# Imports and Extra Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime
from datetime import timedelta
sns.set(style="darkgrid")
import requests
import pprint
import re

In [2]:
time1 = datetime.now()
filename = '../cleaned_flight_data.csv'
random.seed(234)
p = 0.01  # p% of lines
df = pd.read_csv(
         filename,
         header=0, 
         parse_dates = ['FL_DATE','dep_datetime','arr_datetime'],
         skiprows=lambda i: i>0 and random.random() > p
)
time2 = datetime.now()
print('Time cost: ', time2-time1)

/Users/qingyuye/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (77,84,104,105,106,108,110,114,115,116,118,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Time cost:  0:00:41.104616


In [16]:
df.shape

(110935, 121)

In [3]:
# if DEP_DELAY is NaN, drop because useless for our model
df = df.dropna(subset=['DEP_DELAY'])

## Visibility Cleaning and Imputing NaN values

In [4]:
# visibility column cleaning
time1 = datetime.now()

import math
def spacetonan(x):
    if isinstance(x, str) and x != '':
        return float(x)
    elif x == '':
        return np.nan
    return x

# # remove 'V', 'Vs', 's', etc.
df['HOURLYVISIBILITY_origin'] = df['HOURLYVISIBILITY_origin'].str.replace('[^\d(.)]', "")
df['HOURLYVISIBILITY_dest'] = df['HOURLYVISIBILITY_dest'].str.replace('[^\d(.)]', "")

# for the entries with space, fill with nan
df['HOURLYVISIBILITY_origin'] = list(map(spacetonan, df['HOURLYVISIBILITY_origin']))
df['HOURLYVISIBILITY_dest'] = list(map(spacetonan, df['HOURLYVISIBILITY_dest']))

# impute the nan values to mean (~9.4 mi visibility)
np.set_printoptions(threshold=np.nan)
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df['HOURLYVISIBILITY_origin_imp'] = imp_mean.fit_transform(df[['HOURLYVISIBILITY_origin']]).ravel()
df['HOURLYVISIBILITY_dest_imp'] = imp_mean.fit_transform(df[['HOURLYVISIBILITY_dest']]).ravel()

time2 = datetime.now()
print('Time cost: ', time2-time1)

Time cost:  0:00:01.660240


## Parsing "present weather type" and adding column with most salient weather code

In [25]:
# given an entry in the HOURLYPRSENTWEATHERTYPE, give AW xx value
# otherwise return 0
def regex_weather_cond(h_pres_w):
    pattern = re.compile(r'\d{2}')
    if isinstance(h_pres_w, str):
        if re.search(r'\|.*?(([A-Za-z]{2,}:(\d{2})\s)*)\|', h_pres_w):
            string = re.search(r'\|.*?(([A-Za-z]{2,}:(\d{2})\s)*)\|', h_pres_w).group(0)
            match = pattern.findall(string)
        else:
            return 0
    else:
        return 0
    return most_severe_cond(match)

# hard code severe weather condition
# based mostly on avg delay min per weather type
# smoke from Oct 2017 CA forestfire is high cancelled but not delay
def most_severe_cond(lst):
    # freezing rain
    if '64' in lst:
        return 64
    elif '65' in lst:
        return 65
    elif '66' in lst:
        return 66
    # ice pellets
    elif '74' in lst:
        return 74
    elif '75' in lst:
        return 75
    elif '76' in lst:
        return 76
    # thunderstorm with hail
    elif '93' in lst:
        return 93
    # snow
    elif '71' in lst:
        return 71
    elif '72' in lst:
        return 72
    elif '73' in lst:
        return 73
    # fog
    elif '30' in lst:
        return 30
    elif '31' in lst:
        return 31
    elif '32' in lst:
        return 32
    elif '33' in lst:
        return 33
    elif '34' in lst:
        return 34
    # safety return 0 if somehow pass empty list
    elif len(lst) == 0:
        return 0
    # return max element as int from list of strings
    # after those preceding cases, most severe is higher code
    # also not too many cases of combination of drastically different codes 
    else:
        return max(list(map(int, lst)))

In [29]:
# make the present weather type columns into new col
# with most severe weather code selected as integer
time1 = datetime.now()
df['origin_sev_code'] = df['HOURLYPRSENTWEATHERTYPE_origin'].apply(
    lambda x: regex_weather_cond(x))
df['dest_sev_code'] = df['HOURLYPRSENTWEATHERTYPE_dest'].apply(
    lambda x: regex_weather_cond(x))
time2 = datetime.now()
print('Time cost: ', time2-time1)

Time cost:  0:00:00.246934


In [34]:
print(df.origin_sev_code.unique())
print(df.dest_sev_code.unique())

[ 0 91 71 61 62  5 51 31 72 35 92 63 73 90 30 67 95 74 64 81 54 93 33 75
 82 52 65 99  4 18 68]
[ 0 92 71 61 31 51 91 62  5 90 72 54 30 63 74 73 35 67 95 89 18 64 52 68
 65 75 33 93 81  4]


# Binary Classification

Using "DEP_DEL15":
- 1 means delay >= 15
- 0 means delay < 15 (on-time / early)

Goal is to predict this classification. We will use **F1-score** as the primary evaluation metric, since we are concerned with both classes of classificacation: on-time/early vs. delayed. Since the classes are unbalanced, metrics such as accuracy will give a baseline model where we classify all flights as on-time very well (which is misleading since that would not be a useful model). 

Let's start with a **random forest** weather model, where we predict the DEP_DEL15 using the features:
- weather code - origin
- visibilty - origin
- wind speed at origin
- precipitation at origin
- (possibly temperature)

After weather, we will use flight/airport data:
- number of seats
- departure airport (*need encoding/merging - use passenger count*)
- carrier (*need ordinal encoding*)
- departure time (DEP_TIME)
- duration of flight

Later on if we have time:
- cloud cover (need complicated regex)
- time dependence/delay
- other things

http://scikit-learn.org/stable/auto_examples/ensemble/plot_feature_transformation.html#sphx-glr-auto-examples-ensemble-plot-feature-transformation-py

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier

https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html

https://en.wikipedia.org/wiki/Receiver_operating_characteristic

http://scikit-learn.org/stable/modules/model_evaluation.html

http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [37]:
df['DEP_DELAY_GROUP'].describe()
df['DEP_DEL15'].describe()

count    110935.000000
mean          0.176383
std           0.381147
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: DEP_DEL15, dtype: float64

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# models

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [8]:
df.head()

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK    FL_DATE  \
0  2016        1      1            29            5 2016-01-29   
1  2016        1      1            29            5 2016-01-29   
2  2016        2      4            26            2 2016-04-26   
3  2016        2      6            18            6 2016-06-18   
4  2016        2      6            18            6 2016-06-18   

  OP_UNIQUE_CARRIER  OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM  \
0                DL                  19790         DL   N635DL   
1                WN                  19393         WN   N760SW   
2                OO                  20304         OO   N745SK   
3                WN                  19393         WN   N8627B   
4                DL                  19790         DL   N939DN   

             ...              HOURLYPRSENTWEATHERTYPE_dest  \
0            ...                                       NaN   
1            ...                                       NaN   
2            ...                                       NaN   
3            ...                                       NaN   
4            ...                                       NaN   

   HOURLYDRYBULBTEMPF_dest  HOURLYRelativeHumidity_dest  HOURLYWindSpeed_dest  \
0                       35                           64                    10   
1                       56                           72                     3   
2                     44.0                           83                    15   
3                       79                           47                     6   
4                       88                           66                    10   

  HOURLYWindDirection_dest HOURLYWindGustSpeed_dest HOURLYPrecip_dest  \
0                      280                      NaN              0.00   
1                      210                      NaN              0.00   
2                      050                       25              0.00   
3                      190                      NaN              0.00   
4                      250                      NaN              0.00   

   HOURLYAltimeterSetting_dest HOURLYVISIBILITY_origin_imp  \
0                        29.69                        10.0   
1                        30.12                        10.0   
2                        29.93                        10.0   
3                        30.32                        10.0   
4                        30.01                        10.0   

   HOURLYVISIBILITY_dest_imp  
0                       10.0  
1                       10.0  
2                       10.0  
3                       10.0  
4                       10.0  

[5 rows x 123 columns]

In [16]:
df['DEP_TIME']

0         2202.0
1         1955.0
2         1920.0
3          541.0
4          837.0
5         1027.0
6         2031.0
7         1230.0
8         1805.0
9         1536.0
10         610.0
11         555.0
12         952.0
13        1003.0
14         559.0
15        1914.0
16        2018.0
17        1546.0
18        1853.0
19         822.0
20         824.0
21         904.0
22        1855.0
23        1045.0
24         945.0
25         914.0
26         844.0
27         855.0
28         933.0
29         823.0
           ...  
112050    1737.0
112051    1614.0
112052    1458.0
112053     936.0
112054     951.0
112055    1632.0
112056    1035.0
112057    1357.0
112058    1117.0
112059     835.0
112060    1851.0
112061    1855.0
112062    1921.0
112063    1853.0
112064    1439.0
112066    1727.0
112067    1731.0
112068    1715.0
112069    1726.0
112070    1637.0
112072     959.0
112073    1007.0
112074    1123.0
112075    1128.0
112076    1618.0
112077     959.0
112078    1005.0
112079     831

In [17]:
from sklearn import svm

In [18]:
df['number_of_seats']

0         180.0
1         143.0
2          69.0
3         175.0
4         158.0
5          50.0
6         124.0
7         159.0
8         143.0
9         179.0
10        150.0
11         76.0
12        150.0
13         50.0
14        160.0
15        149.0
16        159.0
17        179.0
18        100.0
19        158.0
20        150.0
21        143.0
22         99.0
23        181.0
24         50.0
25        150.0
26        166.0
27        110.0
28        160.0
29        192.0
          ...  
112050    144.0
112051    144.0
112052    144.0
112053    144.0
112054    144.0
112055    144.0
112056    144.0
112057    144.0
112058    144.0
112059    144.0
112060    144.0
112061    144.0
112062    144.0
112063    144.0
112064    144.0
112066    144.0
112067    144.0
112068    144.0
112069    144.0
112070    144.0
112072    144.0
112073    144.0
112074    144.0
112075    144.0
112076    144.0
112077    144.0
112078    144.0
112079    144.0
112080    144.0
112081    144.0
Name: number_of_seats, L

In [22]:
X_column_list = ['DEP_TIME', 'number_of_seats']
y_column = ['DEP_DEL15']
X = df[X_column_list]
y = df[y_column]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf = svm.SVC(kernel='linear', gamma='auto')
clf.fit(X, y)
y_pred = clf.predict(X_test)
metrics.f1_score(y_test, y_pred)

/Users/qingyuye/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
